## Kyoto2006+ データセットを題材にした SVM を用いたの予測モデルの生成と評価

### 準備とデータの読み込み

K-means の時と同様に必要なパッケージを読み込む

In [13]:
# from sklearnex import patch_sklearn
# patch_sklearn()
import pandas
import glob
from sklearn.preprocessing import MinMaxScaler

Kyoto2006+ データセットは CSV 形式でデータが格納されており、それを表形式に変換する。変換後のファイルの各列の名前の順をここで指定する。

In [14]:
column = [
    ["duration", "float32"],
    ["service", "object"],
    ["source_bytes", "uint32"],
    ["destination_bytes", "uint32"],
    ["count", "uint32"],
    ["same_srv_rate", "float32"],
    ["serror_rate", "float32"],
    ["srv_serror_rate", "float32"],
    ["dst_host_count", "uint32"],
    ["dst_host_srv_count", "uint32"],
    ["dst_host_same_src_port_rate", "float32"],
    ["dst_host_serror_rate", "float32"],
    ["dst_host_srv_serror_rate", "float32"],
    ["flag", "object"],
    ["ids_detection", "object"],
    ["malware_detection", "object"],
    ["ashula_detection", "object"],
    ["label", "int8"],
    ["source_ip_address", "object"],
    ["source_port_number", "uint16"],
    ["destination_ip_address", "object"],
    ["destination_port_number", "uint16"],
    ["start_time", "object"],
    ["protocol", "object"]
]
column_names = [i[0] for i in column]
column_types = {i[0]: i[1] for i in column}

データセットを読み込む。時間の短縮のため、2015年2月分のみを用いて演習の手順を説明する。
成果発表では、できるだけデータセット全体を用いて評価してみること。

今回はデータ分析のためのツールである Pandas (https://pandas.pydata.org/) を利用する。

`read_csv` はデータ間がカンマで区切られた txt ファイルからデータを読み込む API である。
第1引数はファイルのパス、`sep`は区切り文字の指定、`header`はtxtファイルの中で列名が入っている行番号、`names`は列名のリストを指定する。CSVファイルには列名は入っていないので `None` を、`names` は `column` のリストを利用する。

In [15]:
files = glob.glob("./dataset/kyoto2006plus/Kyoto2016/2015/02/201502*.txt")
files.sort() # ファイルを時系列順に整列
kyoto_data = pandas.concat([pandas.read_csv(x, sep='\t', header=None, names=column_names, dtype=column_types) for x in files], ignore_index=True)

データの数や平均、分散などの性質を確認するには、`describe` メソッドを利用する。

In [17]:
kyoto_data.describe()

,duration,source_bytes,destination_bytes,count,same_srv_rate,serror_rate,srv_serror_rate,dst_host_count,dst_host_srv_count,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,label,source_port_number,destination_port_number
count,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06
mean,1.317268e+00,5.492566e+04,2.175131e+03,5.175024e+00,4.744613e-01,4.152513e-02,4.192815e-01,3.506128e+01,4.014791e+01,2.828774e-02,8.570371e-02,1.340752e-01,-9.081160e-01,3.344987e+04,2.309937e+03
std,7.953889e+01,8.052808e+06,1.331901e+06,1.023963e+01,4.968409e-01,1.955620e-01,4.526754e-01,4.195258e+01,4.327172e+01,1.617517e-01,2.684495e-01,3.284109e-01,4.189721e-01,2.062878e+04,7.910031e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.220000e+04,2.300000e+01
50%,4.210000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,1.100000e+01,1.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,3.795400e+04,5.300000e+01
75%,1.435795e+00,6.500000e+01,1.180000e+02,4.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,9.200000e+01,9.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,5.122700e+04,4.450000e+02
max,8.392452e+04,2.121764e+09,1.583454e+09,1.000000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.553500e+04,6.553500e+04


それぞれのフローが攻撃かどうかは "label" 列に入っている。

1は正常な通信、その他は不正な通信であり、不正な通信はいくつかの種類(-1:known attack, -2:unknown attack)に分かれている。

label の種類数やそれぞれの label の数を表示するには、値とその値を持つデータの数を出力する `value_counts()` メソッドを利用する。

In [18]:
kyoto_data['label'].value_counts()

label
-1    7513100
 1     362107
-2        557
Name: count, dtype: int64

### 数値化できていないデータの削除
データセットの中には、そのまま用いることは困難な（そのままでは距離を定義することができない）データがある。
例えば、protocol はプロトコルの種別が入っており、量ではないので、異なる種別間の距離は何らかの方法で定義する必要がある。
そのため、学習を実行する前に、データに前処理を施す必要がある。

今回は、そのようなデータを除いて学習を行う。学習に利用する列は以下とする。

In [19]:
use_features = [
    "duration", "source_bytes", "destination_bytes", 
 "count", "same_srv_rate", "serror_rate", "srv_serror_rate","dst_host_count",
  "dst_host_srv_count", "dst_host_same_src_port_rate", "dst_host_serror_rate",
  "dst_host_srv_serror_rate", "source_port_number", "destination_port_number"
]

この列のみを抽出する。

In [20]:
use_data = kyoto_data[use_features]

それぞれの列の値が取り得る値域は大きく異なるため、値域の幅が大きいものに結果が大きく影響してしまう可能性がある。そこで、`sklearn.preprocessing.MinMaxScaler` を利用し、最小値を0、最大値を1に正規化する。

具体的には、MinMaxScaler の `fit_transform` メソッドを利用する。

この返り値は numpy の array なので、Pandas で扱うには Pandas の DataFrame に戻す必要がある。

In [21]:
use_data = pandas.DataFrame(MinMaxScaler().fit_transform(use_data))

describe メソッドで最小値が0, 最大値が1になっていることが確認できる。

In [22]:
use_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06
mean,1.569586e-05,2.588679e-05,1.373662e-06,5.175024e-02,4.744610e-01,4.152513e-02,4.192817e-01,3.506128e-01,4.014791e-01,2.828774e-02,8.570375e-02,1.340752e-01,5.104123e-01,3.524738e-02
std,9.477432e-04,3.795336e-03,8.411361e-04,1.023963e-01,4.968409e-01,1.955620e-01,4.526754e-01,4.195258e-01,4.327172e-01,1.617517e-01,2.684495e-01,3.284109e-01,3.147750e-01,1.206993e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.861601e-01,3.509575e-04
50%,5.016413e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,1.100000e-01,1.600000e-01,0.000000e+00,0.000000e+00,0.000000e+00,5.791409e-01,8.087282e-04
75%,1.710817e-05,3.063489e-08,7.452062e-08,4.000000e-02,1.000000e+00,0.000000e+00,1.000000e+00,9.200000e-01,9.600000e-01,0.000000e+00,0.000000e+00,0.000000e+00,7.816739e-01,6.790265e-03
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


### SVMの適用と結果の確認

今回用いるSVMは教師あり学習である。そのため、データに対してラベルを与える必要がある。なお、攻撃か否かを判断する場合は -1:known attack と-2:unknown attack の区別は不要であるため、"-2"を"-1"に置換する。

In [23]:
label_data = kyoto_data['label']
label_data = label_data.replace(-2, -1)

このデータに対して、SVM を実行する。ここでは例としてrbfカーネル`'rbf'`を指定している。テストデータを全て使用するとかなりの時間がかかるため、ここでは先頭から100000個のみを用いて検証している。

学習には10分程度要する。 ただし、演習の本筋ではないため、実行はスキップして良い。実行結果の例はコメントで用意している。

In [24]:
print(label_data)

0         -1
1         -1
2         -1
3         -1
4         -1
          ..
7875759   -1
7875760   -1
7875761   -1
7875762   -1
7875763   -1
Name: label, Length: 7875764, dtype: int8


In [25]:
from sklearn.svm import SVC
# 線形SVMのインスタンスを生成
model = SVC(kernel='rbf', gamma='auto', random_state=None)

# 学習するデータの数
learn_amounts = 100000

# モデルの学習。fit関数で行う。
model.fit(use_data[:learn_amounts], label_data[:learn_amounts])

# Output
# SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)

SVC(gamma='auto')

学習が完了したら、訓練データそのものに対する精度を確認する。

学習をスキップした場合は、このプログラムもスキップすること。

In [26]:
from sklearn.metrics import accuracy_score

# 訓練データに対する精度
pred_train = model.predict(use_data[:learn_amounts])
accuracy_train = accuracy_score(label_data[:learn_amounts], pred_train[:learn_amounts])
print('訓練データに対する正解率： %.2f' % accuracy_train)

# Output
# 訓練データに対する正解率： 0.97

訓練データに対する正解率： 0.97


以上のように、訓練データに対して、そこそこ高い正解率が得られていると言える。しかし、データの分類を見て見ると、

(こちらも、学習をスキップした場合はプログラムの実行をスキップすること)

In [27]:
label_kinds = [1, -1]
label_dict = {1:'normal.', -1:'attack.'}

# 予測値ごとに学習データを類別する
label_names = [label_data[:learn_amounts][pred_train==x] for x in label_kinds]
for idx, val in enumerate(label_kinds):
    print("predict: {}".format(label_dict[val]))
    print(label_names[idx].value_counts())
    print()
    
# Output
# predict: normal.
# Series([], Name: label, dtype: int64)
# 
# predict: attack.
# -1    96794
#  1     3206
# Name: label, dtype: int64

predict: normal.
Series([], Name: count, dtype: int64)

predict: attack.
label
-1    96794
 1     3206
Name: count, dtype: int64



ほぼ全てのデータを"attack"だと判断していることが分かる。そもそも本来"normal"に分類すべきデータ数が全体の数%しかないのだから、全て"attack"とみなしても、これだけの精度が得られたように見えるのである。
しかし、このような分類では現実においては何の役にも立たないため、改良が必要である。

### 不均衡データへの対策
今回の例のように訓練データの数に偏りがある場合、全てのデータが一方に分類されるという恐れがある。その対策にはいくつかの方法があるが、ここでは使うデータ数を少ない方に合わせる「Under Sampling」を適用する。

In [32]:
# labelを学習用データuse_dataと統合する
use_data['label'] = label_data

# labelの値で類別する
normal_data = use_data[use_data.label == 1]
attack_data = use_data[use_data.label == -1]

# ランダムにサンプリングする(random_state=0より実行毎に値が変わることはない)
sample_amounts = int(len(normal_data)/8)
attack_data_sampled = attack_data.sample(n=sample_amounts, random_state=0)
normal_data_sampled = normal_data.sample(n=sample_amounts, random_state=0)

# 統合する
use_data_sampled = pandas.concat([normal_data_sampled,attack_data_sampled])

# 数の確認
print(['label'])
use_data_sampled['label'].value_counts()

['label']


label
 1    45263
-1    45263
Name: count, dtype: int64

こうすることで、データ数を合わせることができる。
このデータに対して再度SVMを用いて学習を行う。

学習には10分程度要する。一つ前の学習と同様に演習の本筋ではないため、プログラムの実行をスキップしても良い。

In [33]:
from sklearn.svm import SVC
# 線形SVMのインスタンスを生成
model = SVC(kernel='rbf', gamma='auto', random_state=None)

# モデルの学習。fit関数で行う。
model.fit(use_data_sampled.drop('label', axis=1), use_data_sampled['label'])

# Output
# SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)

SVC(gamma='auto')

### 結果の確認
学習が完了したら、まずは訓練データそのものに対する精度を確認する。

学習をスキップした場合は、このプログラムの実行もスキップすること。

In [34]:
from sklearn.metrics import accuracy_score

# 訓練データに対する精度
pred_train = model.predict(use_data_sampled.drop('label', axis=1))
accuracy_train = accuracy_score(use_data_sampled['label'], pred_train)
print('訓練データに対する正解率： %.2f' % accuracy_train)

# Output
# 訓練データに対する正解率： 0.80

訓練データに対する正解率： 0.80


続いてデータの分類を確認する。

こちらも、学習をスキップした場合は、実行をスキップすること。

In [35]:
label_kinds = [1, -1]
label_dict = {1:'normal.', -1:'attack.'}

label_data_sampled = use_data_sampled['label']

label_names = [label_data_sampled.iloc[pred_train==x] for x in label_kinds]
for idx, val in enumerate(label_kinds):
    print("predict: {}".format(label_dict[val]))
    print(label_names[idx].value_counts())
    print()
    
# Output
# predict: normal.
#  1    41446
# -1    14176
# Name: label, dtype: int64

# predict: attack.
# -1    31087
#  1     3817
# Name: label, dtype: int64

predict: normal.
label
 1    41446
-1    14176
Name: count, dtype: int64

predict: attack.
label
-1    31087
 1     3817
Name: count, dtype: int64



### 検証
本演習の目的は、あるフローが与えられた時に正規の通信か不正な通信かを予測する予測モデルを過去のトラフィックデータから生成することである。

ここでは、過去の通信から現在の通信が正規なものか不正なものかを判断するので、時間的な順序を考慮し、データセットを分割し、過去のトラフィックデータで学習させたもので新しい通信の予測がどの程度正しいかどうかの検証を行う。

まずはデータセットのラベルを attack と normal の2種類に変換する関数を用意する。

In [36]:
def get_label(label):
    if label == 1:
        ret = 'normal.'
    else:
        ret = 'attack.'
    return ret
label_dict = {1:'normal.', -1:'attack.'}

データセットを4分割し、過去の3セットで学習し、それより新しい1セットの予測を検証する。

In [37]:
data_len = int(len(use_data)/4)
learn_data = 3

学習に利用するデータセットを準備する

In [38]:
use_data_part = use_data[0:data_len * learn_data]

# labelの値で分離する
normal_data = use_data_part[use_data_part.label == 1]
attack_data = use_data_part[use_data_part.label == -1]

# ランダムにサンプリングする(random_state=0より実行毎に値が変わることはない)
sample_amounts = int(len(normal_data)/6)
attack_data_sampled = attack_data.sample(n=sample_amounts, random_state=0)
normal_data_sampled = normal_data.sample(n=sample_amounts, random_state=0)

# 統合する
use_data_sampled = pandas.concat([normal_data_sampled,attack_data_sampled])

SVMで学習させ、予測を行う。ただし演習の説明では時間短縮のため、学習に用いていなかったデータ全ての予測は行わない。例として、学習に用いていなかったデータの5%だけを予測対象として扱うこととする。

学習と予測には10分程度要する。

In [39]:
from sklearn.svm import SVC
import time
model2 = SVC(kernel='rbf', gamma='auto', random_state=None)

# モデルの学習。fit関数で行う。
model2.fit(use_data_sampled.drop('label', axis=1), use_data_sampled['label'])
# 学習に用いなかったデータの予測を行う。
start = data_len * learn_data # このインデックスまで学習に用いている
length = int((len(use_data)-start)/20) # 予測用データのうち5%だけ、予測対象とする
pred = model.predict(use_data.drop('label', axis=1)[start: start + length]) 

予測が正しいかどうかを検証する。

In [40]:
success = 0
fail = 0
normal = 0
attack = 0
# correct_predict
tp = 0 # 真陽性(true positive) ：　正しく陽性と判定
fn = 0 # 偽陰性(false negative)　：　本当は陽性なのに、陰性と判定
fp = 0 #偽陽性(false positive)　：　本当は陰性なのに、陽性と判定
tn = 0 #真陰性(true negative)　：　正しく陰性と判定
    
for i in range(length):
    predicate = label_dict[pred[i]]
    correct = get_label(kyoto_data['label'][data_len * learn_data + i])
    if predicate == correct:
        # 正常通信と攻撃通信の数を計算
        success += 1
        # 真陽性か真陰性か
        if correct == 'normal.':
            tp += 1
        elif correct == 'attack.':
            tn += 1
    else:
        fail += 1
        # 偽陰性か偽陽性か
        if correct == 'normal.':
            fn += 1
        elif correct == 'attack.':
            fp += 1
    # 正常通信と攻撃通信の数を計算
    if correct == 'normal.':
        normal += 1
    elif correct == 'attack.':
        attack += 1
print("success = {}, failed = {}, normal = {}, attack = {}, unknown = {}".format(success, fail, normal, attack, success + fail - normal - attack))
print("TP = {}, FN = {}, FP = {}, TN = {}".format(tp, fn, fp, tn))

success = 85302, failed = 13145, normal = 2883, attack = 95564, unknown = 0
TP = 2549, FN = 334, FP = 12811, TN = 82753


また、予測が正しいかの検証は以下のプログラムを実行しても確認できる。混同行列と呼ばれる2×2行列の各成分を見ることで、TPの値などを調べられる。
混合行列である`confusion_matrix`から各成分の値を取り出す際は、`ravel()`メソッドを用いれば良い。


$$
    ConfusionMatrix = \left[\begin{array}{c|c} TN & FP \\ \hline FN & TP \\ \end{array}\right]
$$

In [41]:
from sklearn.metrics import confusion_matrix
predicate = pred
correct = label_data[start:start+ length]

c_matrix = confusion_matrix(correct, predicate)
tn, fp, fn, tp = c_matrix.ravel()
print(c_matrix)

[[82753 12811]
 [  334  2549]]


分類問題のモデルを評価する際に使われる代表的な評価指標
1. 正確度(Accuracy): 推定した値と真の値が一致した割合
$$ \frac{TP + TN}{TP + FP + TN + FN} = \frac{正しく判定}{全体}$$
    - FPとFNの重要度について考慮しなくていい場合に使える
2. 適合率(Precision): モデルが陽性と判定した中で、真に陽性だった割合 
$$ \frac{TP}{TP + FP} = \frac{真に陽性}{陽性と判定} $$
    - 明らかに陽性と分かりやすいものだけを見つけたい時
    - FNが発生することを許容できるようなケースで、それでもなおFPがあっては困る場合に使える
3. 再現率(Recall): 真に陽性だったものの中で、モデルが陽性と判定した割合 
$$ \frac{TP}{TP + FN} = \frac{陽性と判定}{真に陽性} $$
    - 怪しいものを全て見つけ出したい時
    - FPが発生することを許容できるケースで、それでもなおFNがあっては困る場合に使える。
4. F-値(F-measure): 適合率と再現率の調和平均
$$ \frac{2}{\frac{1}{Precision} + \frac{1}{Recall}} =  \frac{2 * (Precision * Recall)}{Precision + Recall} $$
    - 正確度と違い、陽性と陰性の出現度合いが極端に異なる場合でも、評価しやすい
5. 特異度(Specificity): 実際に陰性だったものの中で、モデルが陽性と判断した割合
$$ \frac{TN}{FP + TN} $$

In [42]:
print("Accuracy = {}" .format((tp + tn) / (tp + tn + fp + fn) ))
if tp > 0 or fp > 0:
    print("Precision = {}" .format( tp/ (tp + fp) ))
print("Recall = {}" .format(tp/ (tp + fn) ))
print("F-measure = {}" .format(2 * tp / (2 * tp + fn + fp ) ))
print("Specificity = {}" .format(tn/(fp + tn) ))

Accuracy = 0.8664763781527116
Precision = 0.16595052083333334
Recall = 0.8841484564689559
F-measure = 0.2794496519212849
Specificity = 0.8659432422251057


実際には、学習および検証に使うデータの時期を少しずつ変更して何度も評価が行われる。
例えば、利用するデータセットを n 週間分 (n = 1,...) ずらして同様の評価を行うなどが考えられる。

演習の手順説明では`sklearn.svm.SVC`を用いた。このモデルには他にもカーネル関数として `poly` などが用意されている。詳細は http://scikit-learn.org/stable/modules/svm.html#svm-kernels を参照すること。

また、今回のデータセットは量が多いため線形SVMを使うことも考えられる。この場合は学習前にデータセットを近似することで効率を上げることができる。
> For large datasets consider using sklearn.linear_model.LinearSVC or sklearn.linear_model.SGDClassifier instead, possibly after a sklearn.kernel_approximation.Nystroem transformer.

引用: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html